In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("LinearAlgebra")
Pkg.add("Statistics")
Pkg.add("DataFrames")
using CSV
using Statistics
using LinearAlgebra
using DataFrames

  Updating registry at `/srv/julia/pkg/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]


In [2]:
dataset = CSV.read("housingPriceData.csv");


In [3]:
bedrooms=dataset.bedrooms;
bathrooms=dataset.bathrooms;
sqft_living=dataset.sqft_living;
Y=dataset.price;

function scale_features(s)

   μ = mean(s, dims=1);

   σ = std(s, dims=1);

 

   s_norm = (s .- μ) ./ σ;

 

    return (s_norm)

end
bedrooms=scale_features(bedrooms);
bathrooms=scale_features(bathrooms);
sqft_living=scale_features(sqft_living);


In [4]:
trainset = round(0.8*length(bedrooms));
bedrooms_training = bedrooms[1:Int(trainset)];
bedrooms_test = bedrooms[Int(trainset)+1:length(bedrooms)];
bathrooms_training = bathrooms[1:Int(trainset)];
bathrooms_test = bathrooms[Int(trainset)+1:length(bedrooms)];
sqft_living_training = sqft_living[1:Int(trainset)];
sqft_living_test = sqft_living[Int(trainset)+1:length(bedrooms)];
Y_training = Y[1:Int(trainset)];
Y_test = Y[Int(trainset)+1:length(bedrooms)];


In [5]:
m_total = length(bedrooms);
x0_total = ones(m_total);
X_total = cat(x0_total,bedrooms, sqft_living,bedrooms.*bedrooms,sqft_living.*sqft_living,sqft_living.*bedrooms, dims=2)#X = cat(x0_training, bedrooms_training, sqft_living_training,bedrooms_training.*bedrooms_training,sqft_living_training.*sqft_living_training,sqft_living_training.*bedrooms_training, dims=2);
m_training = length(bedrooms_training);
x0_training = ones(m_training);
X_training = cat(x0_training, bedrooms_training, sqft_living_training,bedrooms_training.*bedrooms_training,sqft_living_training.*sqft_living_training,sqft_living_training.*bedrooms_training, dims=2);
m_test = length(bedrooms_test);
x0_test = ones(m_test);
X_test = cat(x0_test, bedrooms_test, sqft_living_test,bedrooms_test.*bedrooms_test,sqft_living_test.*sqft_living_test,sqft_living_test.*bedrooms_test, dims=2); 

In [6]:
function costFunction(X,Y,Theta)
    m = length(Y)
    cost = (1/(2*m))*sum(((X*Theta)-Y).^2)
    return cost
end

costFunction (generic function with 1 method)

In [8]:
Theta = zeros(6,1);


In [9]:
function gradientDescent(X,Y,Theta,lr,numIteration)
    costHistory = zeros(numIteration);
    m =length(Y)
    for iteration in 1:numIteration
        loss = (X*Theta) - Y;
        gradient = (X' * loss)/m;
        Theta = Theta - lr*gradient;
        cost = costFunction(X,Y,Theta);
        costHistory[iteration] = cost;
    end
  
    return Theta , costHistory 
end

gradientDescent (generic function with 1 method)

In [13]:
lr = 0.01
newTheta, costHistory  = gradientDescent(X_training, Y_training, Theta, lr, 1000);
print(newTheta)

[509215.53221488115; -26127.84536765014; 225635.96243779446; 998.3635067757458; 33266.14758691566; -6808.162260400011]

In [14]:
function cal(X,Y,Theta)
    m = length(Y)
    Ybar=mean(Y)*ones(m)
    RMSE=sqrt(sum((1/m)*((X*Theta-Y).^2)))
    R2=1-(sum((X*Theta-Y).^2)/sum((Y-Ybar).^2))    
    return RMSE,R2
end
RMSE ,R2 = cal(X_test,Y_test,newTheta)

(252961.90951947233, 0.5340751827051399)

In [15]:
Y_Pred = X_total*newTheta;

In [16]:
df = DataFrame(Y_Pred)
CSV.write("1b.csv", df)

"1b.csv"